# 📊 Evaluate Vietnamese-English Model

Evaluate and test the VI → EN translation model.

---

## 1. ⚙️ Setup

In [ ]:
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
!pip install -q sentencepiece sacrebleu google-generativeai python-dotenv tqdm pyyaml

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
print('✓ .env created')

## 2. 🔧 Configure Paths

**Nhập trực tiếp đường dẫn đến các files trên Kaggle:**

In [ ]:
# ============================================================
# 📁 NHẬP ĐƯỜNG DẪN TRỰC TIẾP ĐẾN CÁC FILES TRÊN KAGGLE
# ============================================================

# Đường dẫn đến model checkpoint (.pt file)
MODEL_PATH = "/kaggle/input/YOUR_DATASET/best_model.pt"  # THAY ĐỔI

# Đường dẫn đến tokenizer source (Vietnamese - vì VI → EN)
TOKENIZER_SRC_PATH = "/kaggle/input/YOUR_DATASET/tokenizer_vi.model"  # THAY ĐỔI

# Đường dẫn đến tokenizer target (English - vì VI → EN)
TOKENIZER_TGT_PATH = "/kaggle/input/YOUR_DATASET/tokenizer_en.model"  # THAY ĐỔI

# Config file (thường không cần thay đổi)
CONFIG_PATH = "config/config_vi_en.yaml"

# ============================================================
print(f"📦 Model: {MODEL_PATH}")
print(f"📝 Tokenizer VI (src): {TOKENIZER_SRC_PATH}")
print(f"📝 Tokenizer EN (tgt): {TOKENIZER_TGT_PATH}")
print(f"⚙️ Config: {CONFIG_PATH}")

In [ ]:
# Verify files exist
import os

files_to_check = [
    ("Model", MODEL_PATH),
    ("Tokenizer VI", TOKENIZER_SRC_PATH),
    ("Tokenizer EN", TOKENIZER_TGT_PATH),
    ("Config", CONFIG_PATH)
]

all_exist = True
for name, path in files_to_check:
    exists = os.path.exists(path)
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✅ All files found!")
else:
    print("\n❌ Some files not found. Please check the paths above.")

## 3. 📥 Load Model

In [ ]:
import sys
sys.path.insert(0, '.')

from src.evaluate import load_translator

# Load translator trực tiếp từ paths đã nhập
translator = load_translator(
    checkpoint_path=MODEL_PATH,
    vocab_src_path=TOKENIZER_SRC_PATH,
    vocab_tgt_path=TOKENIZER_TGT_PATH,
    config_path=CONFIG_PATH
)

print('✓ Translator loaded successfully!')

## 4. 🧪 Test Translations

In [ ]:
# Test sentences
test_sentences = [
    "Xin chào, bạn khỏe không?",
    "Tôi yêu Việt Nam.",
    "Hôm nay thời tiết rất đẹp.",
    "Cảm ơn bạn rất nhiều.",
    "Tôi đang học tiếng Anh."
]

print("🇻🇳 → 🇬🇧 Vietnamese to English Translations")
print("="*50)

for vn in test_sentences:
    en = translator.translate(vn)
    print(f"\nVI: {vn}")
    print(f"EN: {en}")

## 5. 📝 Interactive Mode

In [ ]:
# Interactive translation (type 'quit' to exit)
print("Interactive VI → EN Translation")
print("Type Vietnamese text and press Enter")
print("Type 'quit' to exit")
print("="*50)

while True:
    text = input("\nVI: ").strip()
    if text.lower() == 'quit':
        break
    translation = translator.translate(text)
    print(f"EN: {translation}")